<a href="https://colab.research.google.com/github/hquoctuan/Introduction_DL/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lap 02 -21110435

In [ ]:
pip install -q lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 39.0 MB/s eta 0:00:00


In [ ]:
#dowload dataset
!gdown 197JiJUIXbXpZylimFqtBz9DeA_dn2BLa
!unzip /content/data_17K.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/train/pure/dc354_silkcut_002_bottom_212.png  
  inflating: data/train/pure/dc354_discolored_000_bottom_062.png  
  inflating: data/train/pure/dc354_broken_004_bottom_228.png  
  inflating: data/train/pure/new_dc317_discolored_002_top_284.png  
  inflating: data/train/pure/new_dc317_discolored_002_bottom_036.png  
  inflating: data/train/pure/dc354_discolored_000_bottom_276.png  
  inflating: data/train/pure/new_dc354_broken_001_top_220.png  
  inflating: data/train/pure/new_dc317_silkcut_001_top_180.png  
  inflating: data/train/pure/new_dc317_discolored_001_bottom_142.png  
  inflating: data/train/pure/new_dc317_silkcut_000_top_216.png  
  inflating: data/train/pure/dc354_discolored_004_bottom_091.png  
  inflating: data/train/pure/new_dc354_broken_001_top_255.png  
  inflating: data/train/pure/dc354_discolored_006_top_137.png  
  inflating: data/train/pure/dc354_discolored_003_bottom_059.png  
  inflating: data/trai

# Demo


In [ ]:
#Import Library
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
import torchmetrics
import torch
import torchvision
from  sklearn.model_selection import train_test_split
import os
import glob
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
#Path to the val dataset
val_path = "/content/data/validation/"
#Path to the train dataset
train_path = "/content/data/train/"
#List of filenames in the training dataset directory
list_ = [name for name in os.listdir(train_path)]

In [ ]:
list_

['silkcut', 'discolored', 'broken', 'pure']

In [ ]:
# Mapping each label name to a unique integer index
label_to_index = {}
for index, label in enumerate(list_):
    label_to_index[label] = index

def get_path(path, labels_list, label_to_index):
    """
    Generates a list of dictionaries with image details, each containing:
    - 'name': the label name
    - 'path': the file path of an image
    - 'label': the integer label corresponding to the name

    Args:
        path (str): Base directory path for image files.
        labels_list (list): List of label names.
        label_to_index (dict): Dictionary mapping label names to unique integers.

    Returns:
        list: List of dictionaries with details for each image.
    """
    paths = []
    for name in labels_list:
        # Get all .png file paths in the directory for the current label
        file_paths = glob.glob(f"{path}{name}/*.png")

        # Append each image's details to paths list
        for file_path in file_paths:
            paths.append({
                "name": name,
                "path": file_path,
                "label": label_to_index[name]
            })
    return paths

# Retrieve paths and labels for training and validation datasets
train_paths = get_path(train_path, list_, label_to_index)
val_paths = get_path(val_path, list_, label_to_index)

In [ ]:
class DataSet(torch.utils.data.Dataset):
    """
    Custom dataset class for loading and transforming images with labels.
    """

    def __init__(self, features, new_size=(224, 224)):
        """
        Initializes the dataset.

        Args:
            features (list): List of dictionaries containing image paths and labels.
            new_size (tuple): Desired size for resizing images. Default is (224, 224).
        """
        super(DataSet, self).__init__()
        self.features = features  # Store dataset features (paths and labels)
        self.resize_transform = torchvision.transforms.Resize(new_size)  # Resize transformation

    def __len__(self):
        """
        Returns:
            int: Number of samples in the dataset.
        """
        return len(self.features)

    def __getitem__(self, idx):
        """
        Retrieves and processes a sample from the dataset by index.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            dict: Dictionary with the transformed image tensor and label.
        """
        paths = self.features[idx]["path"]  # Image file path
        image = torchvision.io.read_image(paths) / 255.0  # Load and normalize image to [0, 1]
        image = self.resize_transform(image)  # Resize image to target dimensions
        label = torch.tensor(self.features[idx]["label"])  # Convert label to tensor
        return {"image": image, "label": label}

# Initialize the dataset with validation paths
Data = DataSet(val_paths)

# Example usage
next(iter(Data))['image'].shape


torch.Size([3, 224, 224])

In [ ]:
class DataModule(L.LightningDataModule):
    """
    PyTorch Lightning data module for loading and managing datasets.
    Handles training, validation, and test data loaders.
    """

    def __init__(self, batch_size, num_workers, train_path, val_path):
        """
        Initializes the data module with batch size, number of workers,
        and paths to training and validation data.

        Args:
            batch_size (int): Number of samples per batch for data loaders.
            num_workers (int): Number of subprocesses for data loading.
            train_path (str): Directory path to training data.
            val_path (str): Directory path to validation data.
        """
        super(DataModule, self).__init__()
        self.train_path = train_path
        self.val_path = val_path
        self.batch_size = batch_size
        self.num_workers = num_workers

    def setup(self, stage=None):
        """
        Sets up the datasets for training, validation, and testing.
        Splits training data for validation and test sets.

        Args:
            stage (str, optional): The stage for setup (e.g., 'fit' or 'test').
        """
        train_paths = get_path(self.train_path, list_, label_to_index)  # Get training paths
        val_paths = get_path(self.val_path, list_, label_to_index)      # Get validation paths
        train_paths, test_paths = train_test_split(train_paths, test_size=0.2, random_state=0)  # Split for testing

        # Create datasets for train, validation, and test sets
        self.train_dataset = DataSet(train_paths)
        self.val_dataset = DataSet(val_paths)
        self.test_dataset = DataSet(test_paths)

    def train_dataloader(self):
        """
        Returns:
            DataLoader: DataLoader for training set with shuffling enabled.
        """
        return torch.utils.data.DataLoader(self.train_dataset, shuffle=True,
                                           batch_size=self.batch_size, num_workers=self.num_workers)

    def val_dataloader(self):
        """
        Returns:
            DataLoader: DataLoader for validation set with shuffling disabled.
        """
        return torch.utils.data.DataLoader(self.val_dataset, shuffle=False,
                                           batch_size=self.batch_size, num_workers=self.num_workers)

    def test_dataloader(self):
        """
        Returns:
            DataLoader: DataLoader for test set with shuffling disabled.
        """
        return torch.utils.data.DataLoader(self.test_dataset, shuffle=False,
                                           batch_size=self.batch_size, num_workers=self.num_workers)

# Instantiate and set up the data module
data_module = DataModule(32, 2, train_path, val_path)
data_module.setup()

# Get a sample batch from the test loader
test_loader = data_module.test_dataloader()
next(iter(test_loader))


{'image': tensor([[[[0.7765, 0.7744, 0.7733,  ..., 0.9271, 0.9313, 0.9333],
           [0.7681, 0.7668, 0.7663,  ..., 0.9248, 0.9258, 0.9263],
           [0.7546, 0.7545, 0.7551,  ..., 0.9212, 0.9169, 0.9151],
           ...,
           [0.9064, 0.9027, 0.8984,  ..., 0.9612, 0.9657, 0.9695],
           [0.9109, 0.9095, 0.9074,  ..., 0.9706, 0.9747, 0.9762],
           [0.9137, 0.9137, 0.9130,  ..., 0.9765, 0.9804, 0.9804]],
 
          [[0.6471, 0.6430, 0.6415,  ..., 0.9443, 0.9451, 0.9451],
           [0.6471, 0.6444, 0.6435,  ..., 0.9491, 0.9473, 0.9451],
           [0.6471, 0.6468, 0.6467,  ..., 0.9567, 0.9508, 0.9451],
           ...,
           [0.9255, 0.9198, 0.9139,  ..., 0.9598, 0.9595, 0.9613],
           [0.9255, 0.9233, 0.9210,  ..., 0.9653, 0.9651, 0.9658],
           [0.9255, 0.9255, 0.9255,  ..., 0.9686, 0.9686, 0.9686]],
 
          [[0.5333, 0.5272, 0.5223,  ..., 0.9443, 0.9336, 0.9255],
           [0.5403, 0.5364, 0.5333,  ..., 0.9471, 0.9351, 0.9255],
           [0.5

In [ ]:
class Model(L.LightningModule):
    """
    A PyTorch Lightning model that uses a ResNet50 backbone for feature extraction
    and a linear classifier for classification. Includes methods for training, validation,
    and testing steps.
    """

    def __init__(self, is_train=False, learning_rate=1e-3, num_target_classes=4):
        """
        Initializes the model with a feature extractor, classifier, and training settings.

        Args:
            is_train (bool): Whether to set the feature extractor to training mode.
            learning_rate (float): The learning rate for the optimizer.
            num_target_classes (int): Number of target classes for classification.
        """
        super(Model, self).__init__()

        # Load the pretrained ResNet50 model
        backbone = torchvision.models.resnet50(weights="DEFAULT")

        # Number of filters in the final fully connected layer of ResNet50
        num_filters = backbone.fc.in_features

        # Remove the last classification layer from ResNet50
        layers = list(backbone.children())[:-1]

        # Define the feature extractor and classifier
        self.feature_extractor = torch.nn.Sequential(*layers)
        self.classifier = torch.nn.Linear(num_filters, num_target_classes)

        # Model training parameters
        self.is_train = is_train
        self.learning_rate = learning_rate

        # Loss function and metric
        self.loss = torch.nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.classification.Accuracy(task="multiclass", num_classes=num_target_classes)

    def forward(self, x):
        """
        Defines the forward pass for the model.

        Args:
            x (Tensor): Input tensor (batch of images).

        Returns:
            Tensor: Output tensor (logits for each class).
        """
        # Set feature extractor to evaluation mode
        self.feature_extractor.eval()

        # Extract features
        if self.is_train:
            representations = self.feature_extractor(x).flatten(1)
        else:
            # Disable gradient calculations for inference
            with torch.no_grad():
                representations = self.feature_extractor(x).flatten(1)

        # Classify the representations
        x = self.classifier(representations)
        return x

    def training_step(self, batch, batch_idx):
        """
        Training step for a batch.

        Args:
            batch (dict): Batch of data, containing images and labels.
            batch_idx (int): Index of the batch.

        Returns:
            Tensor: Training loss for the batch.
        """
        x = batch["image"]
        y = batch["label"]

        # Forward pass
        y_pred = self(x)

        # Calculate loss and accuracy
        loss = self.loss(y_pred, y)
        acc = self.accuracy(y_pred, y)

        # Log metrics
        self.log_dict({'train_loss': loss, 'train_acc': acc}, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        """
        Validation step for a batch.

        Args:
            batch (dict): Batch of data, containing images and labels.
            batch_idx (int): Index of the batch.

        Returns:
            Tensor: Validation loss for the batch.
        """
        x = batch["image"]
        y = batch["label"]

        # Forward pass
        y_pred = self(x)

        # Calculate loss and accuracy
        val_loss = self.loss(y_pred, y)
        val_acc = self.accuracy(y_pred, y)

        # Log metrics
        self.log_dict({'val_loss': val_loss, 'val_acc': val_acc}, prog_bar=True)
        return val_loss

    def test_step(self, batch, batch_idx):
        """
        Test step for a batch.

        Args:
            batch (dict): Batch of data, containing images and labels.
            batch_idx (int): Index of the batch.

        Returns:
            Tensor: Test loss for the batch.
        """
        x = batch["image"]
        y = batch["label"]

        # Forward pass
        y_pred = self(x)

        # Calculate loss and accuracy
        test_loss = self.loss(y_pred, y)
        test_acc = self.accuracy(y_pred, y)

        # Log metrics
        self.log_dict({'test_loss': test_loss, 'test_acc': test_acc}, prog_bar=True)
        return test_loss

    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        """
        Predict step for a batch, used for generating predictions.

        Args:
            batch (dict): Batch of data, containing images.
            batch_idx (int): Index of the batch.
            dataloader_idx (int, optional): Index of the dataloader (for multi-dataloader scenarios).

        Returns:
            None: Implement prediction logic as needed.
        """
        x = batch["image"]
        # Prediction logic here, if needed

    def configure_optimizers(self):
        """
        Sets up the optimizer for the model.

        Returns:
            Optimizer: Adam optimizer with the specified learning rate.
        """
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


In [ ]:
# Create a ModelCheckpoint callback to save the best model during training
model_checkpoint = ModelCheckpoint(
    dirpath='checkpoint/',        # Directory to save the checkpoints
    monitor="val_acc",           # Metric to monitor for checkpointing
    verbose=True,                # Print messages when saving a checkpoint
    mode="max",                  # Save the checkpoint with the maximum validation accuracy
    save_top_k=1                 # Save only the best model
)

# Create an EarlyStopping callback to stop training when the validation accuracy plateaus
early_stopping = EarlyStopping(
    monitor="val_acc",           # Metric to monitor for early stopping
    mode="max",                  # Stop training when the validation accuracy stops improving
    min_delta=1e-4,              # Minimum change to qualify as an improvement
    patience=5                   # Number of epochs with no improvement after which training will be stopped
)

# Combine the callbacks into a list
callbacks = [model_checkpoint, early_stopping]

# Instantiate the model
model = Model()

# Instantiate the DataModule with specified batch size and number of workers
data_module = DataModule(128, 2, train_path, val_path)

# Create the PyTorch Lightning Trainer with specified parameters
trainer = L.Trainer(
    max_epochs=10,               # Maximum number of training epochs
    detect_anomaly=True,         # Enable anomaly detection for debugging
    callbacks=callbacks           # Register the callbacks for training
)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 151MB/s]
INFO: You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
INFO:lightning.pytorch.utilities.rank_zero:You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model,data_module)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name              | Type               | Params | Mode 
-----------------------------------------------------------------
0 | feature_extractor | Sequential         | 23.5 M | train
1 | classifier        | Linear             | 8.2 K  | train
2 | loss              | CrossEntropyLoss   | 0      | train
3 | accuracy          | MulticlassAccuracy | 0      | train
-----------------------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.065    Total estimated model params size (MB)
153       Modules in train mode
0         Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name              | Type               | Params | Mode 
-----------------------------------------------------------------
0 | feature_extractor | Sequential         | 23.5 M | train
1 |

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 90: 'val_acc' reached 0.64846 (best 0.64846), saving model to '/content/checkpoint/epoch=0-step=90.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 0, global step 90: 'val_acc' reached 0.64846 (best 0.64846), saving model to '/content/checkpoint/epoch=0-step=90.ckpt' as top 1


...

INFO: Epoch 7, global step 720: 'val_acc' reached 0.70681 (best 0.70681), saving model to '/content/checkpoint/epoch=7-step=720.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 7, global step 720: 'val_acc' reached 0.70681 (best 0.70681), saving model to '/content/checkpoint/epoch=7-step=720.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 8, global step 810: 'val_acc' reached 0.70796 (best 0.70796), saving model to '/content/checkpoint/epoch=8-step=810.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 8, global step 810: 'val_acc' reached 0.70796 (best 0.70796), saving model to '/content/checkpoint/epoch=8-step=810.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 9, global step 900: 'val_acc' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 9, global step 900: 'val_acc' was not in top 1
INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


## Class Model with Resnet152

In [ ]:
class Model(L.LightningModule):
    """
    A PyTorch Lightning model that uses a ResNet152 backbone for feature extraction
    and a linear classifier for classification. Includes methods for training, validation,
    and testing steps.
    """

    def __init__(self,is_train=False,learning_rate=1e-3, num_target_classes=4):
        """
        Initializes the model with a feature extractor, classifier, and training settings.

        Args:
            learning_rate (float): The learning rate for the optimizer.
            num_target_classes (int): Number of target classes for classification.
        """
        super(Model, self).__init__()

        # Load the pretrained ResNet152 model
        backbone = torchvision.models.resnet152(weights="DEFAULT")

        # Number of filters in the final fully connected layer of ResNet152
        num_filters = backbone.fc.in_features

        # Remove the last classification layer from ResNet152
        layers = list(backbone.children())[:-1]

        # Define the feature extractor and classifier
        self.feature_extractor = torch.nn.Sequential(*layers)
        self.classifier = torch.nn.Linear(num_filters, num_target_classes)

        # Model training parameters
        self.learning_rate = learning_rate

        # Loss function and metric
        self.loss = torch.nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.classification.Accuracy(task="multiclass", num_classes=num_target_classes)

    def forward(self, x):
        """
        Defines the forward pass for the model.

        Args:
            x (Tensor): Input tensor (batch of images).

        Returns:
            Tensor: Output tensor (logits for each class).
        """
        # Extract features
        representations = self.feature_extractor(x).flatten(1)

        # Classify the representations
        x = self.classifier(representations)
        return x

    def training_step(self, batch, batch_idx):
        """
        Training step for a batch.

        Args:
            batch (dict): Batch of data, containing images and labels.
            batch_idx (int): Index of the batch.

        Returns:
            Tensor: Training loss for the batch.
        """
        x = batch["image"]
        y = batch["label"]

        # Forward pass
        y_pred = self(x)

        # Calculate loss and accuracy
        loss = self.loss(y_pred, y)
        acc = self.accuracy(y_pred, y)

        # Log metrics
        self.log_dict({'train_loss': loss, 'train_acc': acc}, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        """
        Validation step for a batch.

        Args:
            batch (dict): Batch of data, containing images and labels.
            batch_idx (int): Index of the batch.

        Returns:
            Tensor: Validation loss for the batch.
        """
        x = batch["image"]
        y = batch["label"]

        # Forward pass
        y_pred = self(x)

        # Calculate loss and accuracy
        val_loss = self.loss(y_pred, y)
        val_acc = self.accuracy(y_pred, y)

        # Log metrics
        self.log_dict({'val_loss': val_loss, 'val_acc': val_acc}, prog_bar=True)
        return val_loss

    def test_step(self, batch, batch_idx):
        """
        Test step for a batch.

        Args:
            batch (dict): Batch of data, containing images and labels.
            batch_idx (int): Index of the batch.

        Returns:
            Tensor: Test loss for the batch.
        """
        x = batch["image"]
        y = batch["label"]

        # Forward pass
        y_pred = self(x)

        # Calculate loss and accuracy
        test_loss = self.loss(y_pred, y)
        test_acc = self.accuracy(y_pred, y)

        # Log metrics
        self.log_dict({'test_loss': test_loss, 'test_acc': test_acc}, prog_bar=True)
        return test_loss

    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        """
        Predict step for a batch, used for generating predictions.

        Args:
            batch (dict): Batch of data, containing images.
            batch_idx (int): Index of the batch.
            dataloader_idx (int, optional): Index of the dataloader (for multi-dataloader scenarios).

        Returns:
            Tensor: Predictions for the input batch.
        """
        x = batch["image"]
        # Forward pass for predictions
        with torch.no_grad():
            y_pred = self(x)
        # Apply softmax to get probabilities
        probabilities = torch.nn.functional.softmax(y_pred, dim=1)

        return probabilities

    def configure_optimizers(self):
        """
        Sets up the optimizer for the model.

        Returns:
            Optimizer: Adam optimizer with the specified learning rate.
        """
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:

# Create a ModelCheckpoint callback to save the best model during training
model_checkpoint = ModelCheckpoint(
    dirpath='checkpoints/',        # Directory to save the checkpoints
    monitor="val_acc",             # Metric to monitor for checkpointing
    verbose=True,                  # Print messages when saving a checkpoint
    mode="max",                    # Save the checkpoint with the maximum validation accuracy
    save_top_k=1                   # Save only the best model
)

# Create an EarlyStopping callback to stop training when the validation accuracy plateaus
early_stopping = EarlyStopping(
    monitor="val_acc",             # Metric to monitor for early stopping
    mode="max",                    # Stop training when the validation accuracy stops improving
    min_delta=1e-4,                # Minimum change to qualify as an improvement
    patience=5                     # Number of epochs with no improvement after which training will be stopped
)

# Combine the callbacks into a list
callbacks = [model_checkpoint, early_stopping]

# Instantiate the model (ensure the Model class is defined correctly to train ResNet152)
model = Model(is_train=True)  # Set is_train=True to ensure the model trains the backbone

# Instantiate the DataModule with specified batch size and number of workers
data_module = DataModule(64, 2, train_path, val_path)

# Create the PyTorch Lightning Trainer with specified parameters
trainer = L.Trainer(
    max_epochs=7,               # Maximum number of training epochs
    detect_anomaly=True,         # Enable anomaly detection for debugging
    callbacks=callbacks           # Register the callbacks for training
)




Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to /root/.cache/torch/hub/checkpoints/resnet152-f82ba261.pth
100%|██████████| 230M/230M [00:01<00:00, 181MB/s]
INFO: You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
INFO:lightning.pytorch.utilities.rank_zero:You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, data_module)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name              | Type               | Params | Mode 
-----------------------------------------------------------------
0 | feature_extractor | Sequential         | 58.1 M | train
1 | classifier        | Linear             | 8.2 K  | train
2 | loss              | CrossEntropyLoss   | 0      | train
3 | accuracy          | MulticlassAccuracy | 0      | train
-----------------------------------------------------------------
58.2 M    Trainable params
0         Non-trainable params
58.2 M    Total params
232.608   Total estimated model params size (MB)
425       Modules in train mode
0         Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name              | Type               | Params | Mode 
-----------------------------------------------------------------
0 | feature_extractor | Sequential         | 58.1 M | train
1 |

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 180: 'val_acc' reached 0.64588 (best 0.64588), saving model to '/content/checkpoints/epoch=0-step=180.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 0, global step 180: 'val_acc' reached 0.64588 (best 0.64588), saving model to '/content/checkpoints/epoch=0-step=180.ckpt' as top 1


...

INFO: Epoch 4, global step 900: 'val_acc' reached 0.72636 (best 0.72636), saving model to '/content/checkpoints/epoch=4-step=900.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 4, global step 900: 'val_acc' reached 0.72636 (best 0.72636), saving model to '/content/checkpoints/epoch=4-step=900.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 5, global step 1080: 'val_acc' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 5, global step 1080: 'val_acc' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 6, global step 1260: 'val_acc' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 6, global step 1260: 'val_acc' was not in top 1
INFO: `Trainer.fit` stopped: `max_epochs=7` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=7` reached.


In [ ]:
model.eval()
#Create list porbilities
all_probabilities =[]
# Each the batch in test_loader
for batch_idx,batch in enumerate(test_loader):
    probabilities = model.predict_step(batch, batch_idx)  # Call predict_step
    all_probabilities.append(probabilities)

all_probabilities = torch.cat(all_probabilities, dim=0)
predicted_labels = torch.argmax(all_probabilities, dim=1)
print(predicted_labels)


tensor([2, 0, 0,  ..., 1, 3, 3])


In [ ]:
#Print probabilitie
print(all_probabilities)

tensor([[2.4330e-02, 3.6882e-01, 4.7348e-01, 1.3337e-01],
        [4.0267e-01, 3.7268e-01, 1.9252e-01, 3.2137e-02],
        [5.4464e-01, 4.3735e-01, 1.7442e-02, 5.6856e-04],
        ...,
        [6.0134e-04, 6.3348e-01, 3.4434e-01, 2.1582e-02],
        [1.1546e-03, 1.1805e-03, 8.8622e-03, 9.8880e-01],
        [1.0722e-03, 4.0547e-03, 7.2736e-03, 9.8760e-01]])
